<a href="https://colab.research.google.com/github/ZFayFay/CNN-DIgits-Recignizer-99.6-/blob/master/Copy_of_Copy_of_CNN_Digits_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [0]:
print(tf.__version__)

1.15.0-rc3


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')

In [0]:
from google.colab import files
uploaded = files.upload()


Saving test.csv to test.csv


In [0]:
import io
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [0]:
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [0]:
Y_train = train["label"]
X_train = train.drop(labels = ["label"],axis =1)

In [0]:
X_train = X_train / 255.0
test = test / 255.0

In [0]:
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
Y_train = to_categorical(Y_train, num_classes = 10)

In [0]:
random_seed = 2
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)

In [0]:
X_train.shape

(37800, 28, 28, 1)

In [0]:
model = Sequential()
model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [0]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 28, 28, 128)       3328      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 28, 28, 128)       409728    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 14, 14, 64)        73792     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 7, 7, 64)         

In [0]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [0]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [0]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [0]:
epochs = 30
batch_size = 86

In [0]:
datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range = 0.1,
        width_shift_range=0.1, 
        height_shift_range=0.1
        )  

datagen.fit(X_train)

In [0]:
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size,callbacks=[learning_rate_reduction])

Epoch 1/30
 - 23s - loss: 0.3941 - acc: 0.8750 - val_loss: 0.0629 - val_acc: 0.9829
Epoch 2/30
 - 21s - loss: 0.1150 - acc: 0.9658 - val_loss: 0.0492 - val_acc: 0.9838
Epoch 3/30
 - 21s - loss: 0.0855 - acc: 0.9755 - val_loss: 0.0295 - val_acc: 0.9912
Epoch 4/30
 - 21s - loss: 0.0730 - acc: 0.9794 - val_loss: 0.0278 - val_acc: 0.9912
Epoch 5/30
 - 21s - loss: 0.0654 - acc: 0.9807 - val_loss: 0.0264 - val_acc: 0.9926
Epoch 6/30
 - 21s - loss: 0.0644 - acc: 0.9819 - val_loss: 0.0282 - val_acc: 0.9929
Epoch 7/30
 - 21s - loss: 0.0603 - acc: 0.9828 - val_loss: 0.0327 - val_acc: 0.9926
Epoch 8/30
 - 21s - loss: 0.0586 - acc: 0.9837 - val_loss: 0.0326 - val_acc: 0.9921
Epoch 9/30
 - 21s - loss: 0.0603 - acc: 0.9841 - val_loss: 0.0237 - val_acc: 0.9929

Epoch 00009: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 10/30
 - 21s - loss: 0.0420 - acc: 0.9882 - val_loss: 0.0171 - val_acc: 0.9952
Epoch 11/30
 - 21s - loss: 0.0393 - acc: 0.9892 - val_loss: 0.0200 - val_acc: 